In [ ]:
# Abigail Kelly
# Assignment 3
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
def calculate_L2_norm(array_numerical, array_analytical):
  # Extract the last column of the numerical and analytical arrays
  u_numerical = array_numerical[:, -1]
  u_analytical = array_analytical[:, -1]

  # Calculate the difference between numerical and analytical solutions
  diff = u_numerical - u_analytical

  # Calculate the L2 norm
  l2_norm = np.linalg.norm(diff) / np.sqrt(len(diff))

  return l2_norm

def calculate_infinity_norm(array_numerical, array_analytical):
  # Extract the last column of the numerical and analytical arrays
  u_numerical = array_numerical[:, -1]
  u_analytical = array_analytical[:, -1]

  # Calculate the difference between numerical and analytical solutions
  diff = u_numerical - u_analytical

  # Calculate the infinity norm
  infinity_norm = np.linalg.norm(diff, ord=np.inf)

  return infinity_norm

In [ ]:
# Question 1 Functions

def q1_euler(max_x, dx, max_t, dt):
  M = int(max_x / dx) + 1
  N = int(max_t / dt) + 1
  x = np.linspace(0, max_x, M)
  t = np.linspace(0, max_t, N)
  kappa = 1
  r = kappa * dt / dx**2

  A = np.zeros((M-2, M-2))
  np.fill_diagonal(A, 1 + 2*r)
  np.fill_diagonal(A[1:], -r)
  np.fill_diagonal(A[:, 1:], -r)
  u = np.zeros((M, N))

  # initial condition
  for i in range(M):
    xi = i * dx
    if 0.3 < xi <= 0.5:
      u[i, 0] = 25 * xi - 7.5
    elif 0.5 < xi <= 0.7:
      u[i, 0] = -25 * xi + 17.5
    else:
      u[i, 0] = 0

  for j in range(N-1):
      b = u[1:M-1, j].copy()

      b[0] += r * (t[j+1] + 1)  # left boundary
      b[-1] += r * (t[j+1]**2 / (t[j+1] + 1))  # right boundary

      u[1:M-1, j+1] = np.linalg.solve(A, b)

      u[0, j+1] = u[1, j+1] - (t[j+1] + 1)  # left boundary
      u[M-1, j+1] = u[M-2, j+1] + (t[j+1]**2 / (t[j+1] + 1))  # right boundary

  return u

def q1_crank(max_x, dx, max_t, dt):
  M = int(max_x / dx)  # Number of spatial divisions
  N = int(max_t / dt)  # Number of time divisions
  x = np.linspace(0, max_x, M+1)
  t = np.linspace(0, max_t, N+1)
  kappa = 1
  r = kappa * dt / (2 * dx**2)

  A = np.diag((1 + r) * np.ones(M-1)) - np.diag(r/2 * np.ones(M-2), 1) - np.diag(r/2 * np.ones(M-2), -1)
  B = np.diag((1 - r) * np.ones(M-1)) + np.diag(r/2 * np.ones(M-2), 1) + np.diag(r/2 * np.ones(M-2), -1)

  C = np.zeros((M+1, N+1))

  # initial conditions
  for i in range(M+1):
    xi = i * dx
    if 0.3 < xi <= 0.5:
      C[i, 0] = 25 * xi - 7.5
    elif 0.5 < xi <= 0.7:
      C[i, 0] = -25 * xi + 17.5
    else:
      C[i, 0] = 0

  for j in range(N):
    b = np.dot(B, C[1:M, j])
    b[0] += r * (t[j+1] + 1)  # left boundary
    b[-1] += r * (t[j+1]**2 / (t[j+1] + 1))  # right boundary

    # Solve the system for the next time step
    C[1:M, j+1] = np.linalg.solve(A, b)
    C[0, j+1] = C[1, j+1] - (t[j+1] + 1)  # left boundary
    C[M, j+1] = C[M-1, j+1] + (t[j+1]**2 / (t[j+1] + 1))  # right boundary

  return C

In [ ]:
# Question 1
max_x = 1
max_t = 1
a_dx = 0.001
h = np.array([1/40, 1/80, 1/160, 1/320, 1/640, 1/1280])  # need to go one step past
tau = np.array([1/20, 1/40, 1/80, 1/160, 1/320, 1/640])  # in order to calculate ROC
crank_analytical = q1_crank(max_x, a_dx, max_t, a_dx)
euler_analytical = q1_euler(max_x, a_dx, max_t, a_dx)
crank_t1 = crank_analytical[:,-1]
euler_t1 = euler_analytical[:,-1]

euler_solutions = []
crank_solutions = []

for i in range(len(tau)):
  dt = tau[i]
  dx = h[i]

  euler = q1_euler(max_x, dx, max_t, dt)
  euler_solutions.append(euler[:,-1])
  crank = q1_crank(max_x, dx, max_t, dt)
  crank_solutions.append(crank[:,-1])


ROC_euler = []
ROC_crank = []

for i in range(1, len(euler_solutions) - 1):
  dx_curr = h[i]
  dx_prev = h[i - 1]
  dx_next = h[i + 1]

  x_curr = np.arange(0, max_x + dx_curr, dx_curr)
  x_prev = np.arange(0, max_x + dx_prev, dx_prev)
  x_next = np.arange(0, max_x + dx_next, dx_next)

  # get common indicies for top of fraction
  top_fraction_common = np.intersect1d(x_prev, x_curr)
  top_prev_indices = top_fraction_common * int(max_x / dx_prev)
  top_curr_indices = top_fraction_common * int(max_x / dx_curr)

  # get common indicies for bottom of fraction
  bot_fraction_common = np.intersect1d(x_curr, x_next)
  bot_curr_indices = bot_fraction_common * int(max_x / dx_curr)
  bot_next_indices = bot_fraction_common * int(max_x / dx_next)

  top_prev_term_euler = []
  top_curr_term_euler = []
  bot_curr_term_euler = []
  bot_next_term_euler = []
  top_prev_term_crank = []
  top_curr_term_crank = []
  bot_curr_term_crank = []
  bot_next_term_crank = []

  for j in top_prev_indices:
    top_prev_term_euler.append(euler_solutions[i - 1][int(j)])
    top_prev_term_crank.append(crank_solutions[i - 1][int(j)])

  for j in top_curr_indices:
    top_curr_term_euler.append(euler_solutions[i][int(j)])
    top_curr_term_crank.append(crank_solutions[i][int(j)])

  for j in bot_curr_indices:
    bot_curr_term_euler.append(euler_solutions[i][int(j)])
    bot_curr_term_crank.append(crank_solutions[i][int(j)])

  for j in bot_next_indices:
    bot_next_term_euler.append(euler_solutions[i + 1][int(j)])
    bot_next_term_crank.append(crank_solutions[i + 1][int(j)])

  top_prev_term_arr_euler = np.array(top_prev_term_euler)
  top_curr_term_arr_euler = np.array(top_curr_term_euler)
  bot_curr_term_arr_euler = np.array(bot_curr_term_euler)
  bot_next_term_arr_euler = np.array(bot_next_term_euler)
  top_prev_term_arr_crank = np.array(top_prev_term_crank)
  top_curr_term_arr_crank = np.array(top_curr_term_crank)
  bot_curr_term_arr_crank = np.array(bot_curr_term_crank)
  bot_next_term_arr_crank = np.array(bot_next_term_crank)

  ROC_euler.append(np.log2(np.linalg.norm(top_prev_term_arr_euler - top_curr_term_arr_euler) / np.linalg.norm(bot_curr_term_arr_euler - bot_next_term_arr_euler)))
  ROC_crank.append(np.log2(np.linalg.norm(top_prev_term_arr_crank - top_curr_term_arr_crank) / np.linalg.norm(bot_curr_term_arr_crank - bot_next_term_arr_crank)))

crank_l2 = []
euler_l2 = []
for i in range(len(euler_solutions)):
  crank_shared = []
  euler_shared = []
  euler_anal_shared = []
  crank_anal_shared = []

  dx_curr = h[i]
  x_curr = np.arange(0, max_x + dx_curr, dx_curr)   # spatial steps for euler/crank
  x_anal = np.arange(0, max_x + a_dx, a_dx)       # spatial steps for analytical
  common = np.intersect1d(x_curr, x_anal)           # find spatial stamps in common
  ec_indices = common * int(max_x / dx_curr)
  anal_indices = common * int(max_x / a_dx)

  for j in ec_indices:
    euler_shared.append(euler_solutions[i][int(j)])
    crank_shared.append(crank_solutions[i][int(j)])

  for j in anal_indices:
    euler_anal_shared.append(euler_t1[int(j)])
    crank_anal_shared.append(crank_t1[int(j)])

  c_estimate = np.array(crank_shared)
  c_analytical = np.array(crank_anal_shared)
  e_estimate = np.array(euler_shared)
  e_analytical = np.array(euler_anal_shared)

  crank_l2.append(np.linalg.norm(c_analytical - c_estimate))
  euler_l2.append(np.linalg.norm(e_analytical - e_estimate))

print(f'h            tau                     Backward-Euler               Crank-Nicolson')
print(f'====================================================================================')
print(f'                                 L2 error        ROC            L2 error       ROC')
print(f'------------------------------------------------------------------------------------')
print(f'{h[0]:.2e}    {tau[0]:.2e}              {euler_l2[0]:.2e}       -              {crank_l2[0]:.2e}        -')
print(f'{h[1]:.2e}    {tau[1]:.2e}              {euler_l2[1]:.2e}   {ROC_euler[0]:.2e}           {crank_l2[1]:.2e}   {ROC_crank[0]:.2e}')
print(f'{h[2]:.2e}    {tau[2]:.2e}              {euler_l2[2]:.2e}   {ROC_euler[1]:.2e}           {crank_l2[2]:.2e}   {ROC_crank[1]:.2e}')
print(f'{h[3]:.2e}    {tau[3]:.2e}              {euler_l2[3]:.2e}   {ROC_euler[2]:.2e}           {crank_l2[3]:.2e}   {ROC_crank[2]:.2e}')
print(f'{h[4]:.2e}    {tau[4]:.2e}              {euler_l2[4]:.2e}   {ROC_euler[3]:.2e}           {crank_l2[4]:.2e}   {ROC_crank[3]:.2e}')

h            tau                     Backward-Euler               Crank-Nicolson
                                 L2 error        ROC            L2 error       ROC
------------------------------------------------------------------------------------
2.50e-02    5.00e-02              5.36e-02       -              3.27e-01        -
1.25e-02    2.50e-02              2.57e-02   9.95e-01           1.96e-01   2.26e-01
6.25e-03    1.25e-02              1.18e-02   9.93e-01           1.56e-01   7.26e-02
3.13e-03    6.25e-03              4.76e-03   9.89e-01           1.44e-01   2.31e-02
1.56e-03    3.13e-03              1.26e-03   9.80e-01           1.40e-01   7.82e-03


In [ ]:
# Question 2 Functions

def q2_euler(max_x, dx, max_t, dt):
    M = int(max_x / dx) + 1  # Number of spatial steps
    N = int(max_t / dt) + 1  # Number of time steps
    x = np.linspace(0, max_x, M)
    t = np.linspace(0, max_t, N)
    kappa = 1
    r = kappa * dt / dx**2

    A = np.zeros((M-2, M-2))
    np.fill_diagonal(A, 1 + 2*r)
    np.fill_diagonal(A[1:], -r)
    np.fill_diagonal(A[:, 1:], -r)
    u = np.zeros((M, N))

    for i in range(M):
      xi = i * dx
      if 0 <= xi < 0.25:
        u[i, 0] = 0
      elif 0.25 <= xi < 0.75:
        u[i, 0] = 1
      elif 0.75 <= xi <= 1:
        u[i, 0] = 0

    for j in range(N-1):
      b = u[1:M-1, j].copy()

      b[0] += r * (np.e**-t[j+1])  # left boundary
      b[-1] += r * (np.e**-t[j+1])  # right boundary

      u[1:M-1, j+1] = np.linalg.solve(A, b)

      u[0, j+1] = u[1, j+1] - (np.e**-t[j+1])  # left boundary
      u[M-1, j+1] = u[M-2, j+1] + (np.e**-t[j+1])  # right boundary

    return u

def q2_crank(max_x, dx, max_t, dt):
  M = int(max_x / dx)  # Number of spatial divisions
  N = int(max_t / dt)  # Number of time divisions
  x = np.linspace(0, max_x, M+1)
  t = np.linspace(0, max_t, N+1)
  kappa = 1
  r = kappa * dt / (2 * dx**2)

  A = np.diag((1 + r) * np.ones(M-1)) - np.diag(r/2 * np.ones(M-2), 1) - np.diag(r/2 * np.ones(M-2), -1)
  B = np.diag((1 - r) * np.ones(M-1)) + np.diag(r/2 * np.ones(M-2), 1) + np.diag(r/2 * np.ones(M-2), -1)

  C = np.zeros((M+1, N+1))

  for i in range(M+1):
    xi = i * dx
    if 0 <= xi < 0.25:
      C[i, 0] = 0
    elif 0.25 <= xi < 0.75:
      C[i, 0] = 1
    elif 0.75 <= xi <= 1:
      C[i, 0] = 0

  for j in range(N):
      b = np.dot(B, C[1:M, j])
      b[0] += r * (np.e**-t[j+1])  # left boundary
      b[-1] += r * (np.e**-t[j+1]) # right boundary

      # Solve the system for the next time step
      C[1:M, j+1] = np.linalg.solve(A, b)
      C[0, j+1] = C[1, j+1] - (np.e**-t[j+1])  # left boundary
      C[M, j+1] = C[M-1, j+1] + (np.e**-t[j+1])  # right boundary

  return C

In [ ]:
# Question 2
max_x = 1
max_t = 1
a_dx = 0.001
h = np.array([1/40, 1/80, 1/160, 1/320, 1/640, 1/1280])  # need to go one step past
tau = np.array([1/20, 1/40, 1/80, 1/160, 1/320, 1/640])  # in order to calculate ROC
euler_solutions = []
crank_solutions = []
crank_analytical = q2_crank(max_x, a_dx, max_t, a_dx)
euler_analytical = q2_euler(max_x, a_dx, max_t, a_dx)
crank_t1 = crank_analytical[:,-1]
euler_t1 = euler_analytical[:,-1]

for i in range(len(tau)):
  dt = tau[i]
  dx = h[i]

  euler = q2_euler(max_x, dx, max_t, dt)
  crank = q2_crank(max_x, dx, max_t, dt)
  euler_solutions.append(euler[:,-1])
  crank_solutions.append(crank[:,-1])

ROC_euler = []
ROC_crank = []

for i in range(1, len(euler_solutions) - 1):
  dx_curr = h[i]
  dx_prev = h[i - 1]
  dx_next = h[i + 1]

  x_curr = np.arange(0, max_x + dx_curr, dx_curr)
  x_prev = np.arange(0, max_x + dx_prev, dx_prev)
  x_next = np.arange(0, max_x + dx_next, dx_next)

  # get common indicies for top of fraction
  top_fraction_common = np.intersect1d(x_prev, x_curr)
  top_prev_indices = top_fraction_common * int(max_x / dx_prev)
  top_curr_indices = top_fraction_common * int(max_x / dx_curr)

  # get common indicies for bottom of fraction
  bot_fraction_common = np.intersect1d(x_curr, x_next)
  bot_curr_indices = bot_fraction_common * int(max_x / dx_curr)
  bot_next_indices = bot_fraction_common * int(max_x / dx_next)

  top_prev_term_euler = []
  top_curr_term_euler = []
  bot_curr_term_euler = []
  bot_next_term_euler = []
  top_prev_term_crank = []
  top_curr_term_crank = []
  bot_curr_term_crank = []
  bot_next_term_crank = []

  for j in top_prev_indices:
    top_prev_term_euler.append(euler_solutions[i - 1][int(j)])
    top_prev_term_crank.append(crank_solutions[i - 1][int(j)])

  for j in top_curr_indices:
    top_curr_term_euler.append(euler_solutions[i][int(j)])
    top_curr_term_crank.append(crank_solutions[i][int(j)])

  for j in bot_curr_indices:
    bot_curr_term_euler.append(euler_solutions[i][int(j)])
    bot_curr_term_crank.append(crank_solutions[i][int(j)])

  for j in bot_next_indices:
    bot_next_term_euler.append(euler_solutions[i + 1][int(j)])
    bot_next_term_crank.append(crank_solutions[i + 1][int(j)])

  top_prev_term_arr_euler = np.array(top_prev_term_euler)
  top_curr_term_arr_euler = np.array(top_curr_term_euler)
  bot_curr_term_arr_euler = np.array(bot_curr_term_euler)
  bot_next_term_arr_euler = np.array(bot_next_term_euler)
  top_prev_term_arr_crank = np.array(top_prev_term_crank)
  top_curr_term_arr_crank = np.array(top_curr_term_crank)
  bot_curr_term_arr_crank = np.array(bot_curr_term_crank)
  bot_next_term_arr_crank = np.array(bot_next_term_crank)

  ROC_euler.append(np.log2(np.linalg.norm(top_prev_term_arr_euler - top_curr_term_arr_euler) / np.linalg.norm(bot_curr_term_arr_euler - bot_next_term_arr_euler)))
  ROC_crank.append(np.log2(np.linalg.norm(top_prev_term_arr_crank - top_curr_term_arr_crank) / np.linalg.norm(bot_curr_term_arr_crank - bot_next_term_arr_crank)))

crank_l2 = []
euler_l2 = []
for i in range(len(euler_solutions)):
  crank_shared = []
  euler_shared = []
  euler_anal_shared = []
  crank_anal_shared = []

  dx_curr = h[i]
  x_curr = np.arange(0, max_x + dx_curr, dx_curr)   # spatial steps for euler/crank
  x_anal = np.arange(0, max_x + a_dx, a_dx)       # spatial steps for analytical
  common = np.intersect1d(x_curr, x_anal)           # find spatial stamps in common
  ec_indices = common * int(max_x / dx_curr)
  anal_indices = common * int(max_x / a_dx)

  for j in ec_indices:
    euler_shared.append(euler_solutions[i][int(j)])
    crank_shared.append(crank_solutions[i][int(j)])

  for j in anal_indices:
    euler_anal_shared.append(euler_t1[int(j)])
    crank_anal_shared.append(crank_t1[int(j)])

  c_estimate = np.array(crank_shared)
  c_analytical = np.array(crank_anal_shared)
  e_estimate = np.array(euler_shared)
  e_analytical = np.array(euler_anal_shared)

  crank_l2.append(np.linalg.norm(c_analytical - c_estimate))
  euler_l2.append(np.linalg.norm(e_analytical - e_estimate))

print(f'h            tau                     Backward-Euler               Crank-Nicolson')
print(f'====================================================================================')
print(f'                                 L2 error        ROC            L2 error       ROC')
print(f'------------------------------------------------------------------------------------')
print(f'{h[0]:.2e}    {tau[0]:.2e}              {euler_l2[0]:.2e}       -              {crank_l2[0]:.2e}        -')
print(f'{h[1]:.2e}    {tau[1]:.2e}              {euler_l2[1]:.2e}   {ROC_euler[0]:.2e}           {crank_l2[1]:.2e}   {ROC_crank[0]:.2e}')
print(f'{h[2]:.2e}    {tau[2]:.2e}              {euler_l2[2]:.2e}   {ROC_euler[1]:.2e}           {crank_l2[2]:.2e}   {ROC_crank[1]:.2e}')
print(f'{h[3]:.2e}    {tau[3]:.2e}              {euler_l2[3]:.2e}   {ROC_euler[2]:.2e}           {crank_l2[3]:.2e}   {ROC_crank[2]:.2e}')
print(f'{h[4]:.2e}    {tau[4]:.2e}              {euler_l2[4]:.2e}   {ROC_euler[3]:.2e}           {crank_l2[4]:.2e}   {ROC_crank[3]:.2e}')

h            tau                     Backward-Euler               Crank-Nicolson
                                 L2 error        ROC            L2 error       ROC
------------------------------------------------------------------------------------
2.50e-02    5.00e-02              8.70e-03       -              3.65e-01        -
1.25e-02    2.50e-02              4.24e-03   7.23e-01           2.78e-01   -2.94e-03
6.25e-03    1.25e-02              1.98e-03   6.49e-01           2.75e-01   1.74e-04
3.13e-03    6.25e-03              8.49e-04   5.89e-01           2.75e-01   1.09e-04
1.56e-03    3.13e-03              2.90e-04   5.49e-01           2.75e-01   5.46e-05
